In [195]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer ## Handling Missing Values
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import OrdinalEncoder 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [196]:
df=pd.read_csv(r"D:\DiamondPricesPredection\notebooks\data\gemstone.csv")
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


### Model Training:-

In [197]:
df=df.drop(labels=['id'],axis=1)

In [198]:
X=df.drop(labels=['price'],axis=1)
Y=df["price"]

In [199]:
Y

0         13619
1         13387
2          2772
3           666
4         14453
          ...  
193568     1130
193569     2874
193570     3036
193571      681
193572     2258
Name: price, Length: 193573, dtype: int64

###  Segregating numerical and categorical variables

In [200]:
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

### Define the custom ranking for each ordinal variable

In [201]:
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [202]:
num_pipeline=Pipeline(
    steps=[
        
            ('imputer',SimpleImputer(strategy='median')),
            ('scaler',StandardScaler())
            
    ]
)

cat_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('OrdinalEncoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scale',StandardScaler())
        
    ]
)


In [203]:
preprocessor=ColumnTransformer(transformers=[
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols),
])

In [204]:
preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')),
                                ('cat_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('OrdinalEncoder',
                                                  OrdinalEncoder(categories=[['Fair',
                                                                              'Good',
                                                                              'Very '
                                                                              'Good',
                                                                              'Premium',
                                                                              'Ideal'],
                                                                             ['D',
                                                                              'E',
                                                                              'F',
                                                                              'G',
                                                                              'H',
                                                                              'I',
                                                                              'J'],
                                                                             ['I1',
                                                                              'SI2',
                                                                              'SI1',
                                                                              'VS2',
                                                                              'VS1',
                                                                              'VVS2',
                                                                              'VVS1',
                                                                              'IF']])),
                                                 ('scale', StandardScaler())]),
                                 Index(['cut', 'color', 'clarity'], dtype='object'))])

#### Train test split

In [205]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.20,random_state=42)

In [206]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [207]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-1.016395,-0.204317,0.402608,-1.202472,-1.187395,-1.194148,-0.132842,-0.936018,-0.648950
1,0.882396,0.720758,-0.118536,0.985177,0.941823,1.036109,-0.132842,-0.320002,0.017052
2,1.529711,0.350728,-1.160823,1.426308,1.394848,1.441611,0.872563,1.528047,0.017052
3,1.896523,0.073206,0.923751,1.741402,1.711965,1.702290,-0.132842,1.528047,-1.314953
4,0.450852,1.738340,1.444895,0.562052,0.525040,0.703019,-2.143651,0.912031,0.017052


### Model Training


In [208]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

In [209]:
models = {     
    "Linear Regression": LinearRegression(),
    'Ridge Regression':Ridge(),
    'Lasso Regression':Lasso(),
    'Elasticnet Regression':ElasticNet(),
    
    
    # "Random Forest": RandomForestRegressor(),
    # "Decision Tree": DecisionTreeRegressor(),
    # "XGBRegressor": XGBRegressor(),
            }

In [210]:
params={
    "Linear Regression":{},
    
    'Ridge Regression':{},
    
    'Lasso Regression':{},
    'Elasticnet Regression':{},
    
    
#     "Decision Tree": {
#         # 'splitter':['best','random'],
#         'max_features':['sqrt','log2'],
#     },
#     "Random Forest":{
#         'criterion':['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],

#         'max_features':['sqrt','log2',None],
#         'n_estimators': [8,16,32,64,128,256]
#     },
    
#     "XGBRegressor":{
#         'learning_rate':[.1,.01,.05,.001],
#         'n_estimators': [8,16,32,64,128,256]
#     } 
}

In [211]:
from sklearn.model_selection import GridSearchCV
def get_best_model(X_train,y_train,X_test,y_test,model_dict,params_,base_accuracy=0.7):
    models = model_dict.keys()
    for model in models:
        print(f'Initial base accuracy is {base_accuracy}')
        print(20*'*',"Estimator: ",model,20*'*')
        gs = GridSearchCV(estimator=model_dict[model],param_grid=params_[model],cv=3)
        gs.fit(X_train,y_train)
        print(f'Best parameters are :{gs.best_params_} With R2 score:{gs.best_score_}')
        
        regressor = model_dict[model].set_params(**gs.best_params_)
        regressor.fit(X_train,y_train)

        y_train_pred = regressor.predict(X_train)
        y_test_pred = regressor.predict(X_test)
        
        train_model_score = r2_score(y_train, y_train_pred)

        test_model_score = r2_score(y_test, y_test_pred)
        
        if test_model_score >= base_accuracy:
            print('Acceptabel model found!')
            base_accuracy = test_model_score
            best_model = {model:regressor}
        else:
            print('Model rejected\n',30*'__')
            
    
    print(f"Best model is:{list(best_model.keys())} with R2 Score: {round(test_model_score,2)}")        
    return best_model

In [212]:
get_best_model(X_train,y_train,X_test,y_test,model_dict=models,params_=params)

Initial base accuracy is 0.7
******************** Estimator:  Linear Regression ********************


Best parameters are :{} With R2 score:0.9364503659912109
Acceptabel model found!
Initial base accuracy is 0.9372975155452961
******************** Estimator:  Ridge Regression ********************
Best parameters are :{} With R2 score:0.9364495711536139
Model rejected
 ____________________________________________________________
Initial base accuracy is 0.9372975155452961
******************** Estimator:  Lasso Regression ********************
Best parameters are :{} With R2 score:0.9364978596601053
Model rejected
 ____________________________________________________________
Initial base accuracy is 0.9372975155452961
******************** Estimator:  Elasticnet Regression ********************
Best parameters are :{} With R2 score:0.8548667975844465
Model rejected
 ____________________________________________________________
Best model is:['Linear Regression'] with R2 Score: 0.85


{'Linear Regression': LinearRegression()}

In [213]:
models=LinearRegression()

In [215]:
models.fit(X_train,y_train)

LinearRegression()

In [218]:
y_pred=models.predict(X_test)

In [219]:
print(r2_score(y_pred,y_test))

0.9323134642469872


In [221]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [223]:
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error

In [224]:
## Train multiple models
## Model Ecaluation
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')


LinearRegression
Model Training Performance
RMSE: 1006.6009861315857
MAE: 671.5856392794417
R2 score 93.7297515545296


Lasso
Model Training Performance
RMSE: 1006.8715806390857
MAE: 672.8634885961557
R2 score 93.72637996466202


Ridge
Model Training Performance
RMSE: 1006.6062314346223
MAE: 671.6137412324215
R2 score 93.72968620700986


Elasticnet
Model Training Performance
RMSE: 1535.594391819253
MAE: 1063.329773546082
R2 score 85.40772247198525


